In [41]:
import os
import cv2
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Convolution2D,Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras import optimizers
from keras.models import Model,load_model

from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [42]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [44]:
import glob

root_path = '/content/drive/My Drive/Smart Health/Assignment_3/DS/'
data_X = []
data_Y = []
err_else = 0
for name in glob.glob(root_path+'*'):
  tmp_class_num = None
  if 'Abdomen' in name:
    tmp_class_num = 0
  elif 'Head' in name:
    tmp_class_num = 1
      # print "Not found!"
  elif 'Chest' in name:
    tmp_class_num = 2    
  each_file_path = name
  img_arr = cv2.imread(each_file_path, cv2.IMREAD_GRAYSCALE)
  resized_arr = cv2.resize(img_arr, (32, 32))
  # print(type(resized_arr))
  if err_else == 0:
    data_X.append(resized_arr)
    data_Y.append(tmp_class_num)
  else:
    print("Break error: ", eachFile)
    break


In [45]:
print(type(data_X), len(data_X), len(data_X[0]), len(data_X[1]))
data_X = np.array(data_X)
data_Y = np.array(data_Y)


<class 'list'> 1500 32 32


In [46]:
print(data_X.shape,"   ",data_Y.shape)
data_X[0]

(1500, 32, 32)     (1500,)


array([[101, 101, 101, ..., 101, 101, 101],
       [101, 101, 101, ..., 101, 101, 101],
       [101, 101, 101, ..., 101, 101, 101],
       ...,
       [101, 101, 101, ..., 101, 101, 101],
       [101, 101, 101, ..., 101, 101, 101],
       [101, 101, 101, ..., 101, 101, 101]], dtype=uint8)

In [ ]:
data_X[1]

In [ ]:
data_X = data_X / 255

In [ ]:
data_X[0]

In [ ]:
data_X[1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data_X, data_Y, test_size=0.3, random_state=42)

In [ ]:
c_a = 0
c_b = 0
c_c = 0
for i in Y_train:
  if i==0:
    c_a = c_a +1
  
  if i==1:
    c_b = c_b +1
  
  if i==2:
    c_c = c_c +1

In [ ]:
print(c_a,"   ",c_b,"  ",c_c)

In [ ]:
plt.figure(figsize = (10, 10))
plt.imshow(data_X[0], cmap='gray')
plt.title(data_Y[0])

In [ ]:
activationFunction = 'relu'
def getModel():
    model = Sequential()
    
    model.add(Conv2D(64, kernel_size = (5, 5), activation=activationFunction, input_shape=(32, 32, 1)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    model.add(Conv2D(48, kernel_size=(3,3), activation=activationFunction))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))
    
    model.add(Conv2D(20, kernel_size=(3,3), activation=activationFunction))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))
    
    model.add(Flatten())
    model.add(Dense(100, activation=activationFunction, name ='feature_dense'))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

model_train = getModel()
# model_test = getModel()

In [ ]:
def showResults(test, pred):
    target_names = ['A', 'C', 'H']
    print(classification_report(test, pred, target_names=target_names))
    accuracy = accuracy_score(test, pred)
    precision=precision_score(test, pred, average='weighted')
    f1Score=f1_score(test, pred, average='weighted') 
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1Score))
    cm=confusion_matrix(test, pred)
    print(cm)

In [ ]:
X_train = np.expand_dims(X_train, axis = 3)
X_test = np.expand_dims(X_test, axis = 3)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
epoch_num = 7

In [ ]:
# y_train_cat = to_categorical(Y_train)
# y_test_cat = to_categorical(Y_test)

In [ ]:
history_train = model_train.fit(X_train, Y_train, 
                    epochs=epoch_num, 
                    batch_size = 64)  

In [ ]:
intermediate_layer_train = Model(inputs=model_train.input,
                                 outputs=model_train.get_layer('feature_dense').output)

intermediate_layer_train.summary()

In [ ]:
XTrainFeature = intermediate_layer_train.predict(X_train)
XTestFeature = intermediate_layer_train.predict(X_test)

In [ ]:
print(XTestFeature.shape,"   ",XTrainFeature.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
neigh = KNeighborsClassifier()
k_range = [3, 5, 7, 9]
param_grid = dict(n_neighbors=k_range)
grid = GridSearchCV(neigh, param_grid, cv=5, scoring='accuracy')

In [ ]:
grid.fit(XTrainFeature, Y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.cv_results_

In [ ]:
import pickle
pkl_knn = "/content/drive/My Drive/Smart Health/publicID-KNN.pkl"
pkl_rf = "/content/drive/My Drive/Smart Health/publicID-RF.pkl"
with open(pkl_knn, 'wb') as file:
  pickle.dump(grid, file)

In [ ]:
with open(pkl_knn, 'rb') as file:
  grid1 = pickle.load(file)
x = grid1.predict(XTestFeature)
showResults(Y_test, x)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf_model = RandomForestClassifier()

In [ ]:
# Create the random grid
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 50, stop = 200, num = 10)],
               'max_depth': [int(x) for x in np.linspace(5, 25, num = 5)],
               'bootstrap': [True, False]
               }

In [ ]:
rf_random = RandomizedSearchCV(estimator =rf_model, 
                               param_distributions = random_grid, 
                               n_iter = 25, cv = 5)
# Fit the random search model
rf_random.fit(XTrainFeature, Y_train)

In [ ]:
rf_random.best_params_

In [ ]:
with open(pkl_rf, 'wb') as file:
  pickle.dump(rf_random, file)

In [ ]:
with open(pkl_rf, 'rb') as file:
  rf_random1 = pickle.load(file)
predictions = rf_random1.predict(XTestFeature)
showResults(Y_test, predictions)